# Text CNN interpretability (for 'model understanding exploration') using TX-Ray
### paper: https://arxiv.org/abs/1912.00982
![TX-Ray overview](https://drive.google.com/uc?id=1YgUjLdApcL4YwBJoLn6otGnqX9yiZQ_c)
### video: https://www.youtube.com/watch?v=iA_QK6KhA5s - more usecases than in this tutorial
+ TX-Ray is based on activation maximization principal (2009 Erhan et al.). That is, for each neuron, record which features the neuron prefers, i.e. maximally activates on.
+ for each neuron we want to interpret, we record the features (words) that maximally activate that neuron. Note: activation maximization in vision NNs is more complicated, see https://distill.pub/2017/feature-visualization.

## <font color='LawnGreen'>Goals</font>:
+ learn how to explore and interpret activations in NNs using 'activation maximization' principles
+ learn how to extract activations via forward_hooks
+ exercise how to usefully interpret and visualize activation behaviour
+ exercise how to prune activations -- advanced
+ think about neuron/ filter redundancy, specialization, generalization - advanced
+ Overall: explore/ develop ideas towards 'model understanding' -- see https://arxiv.org/abs/1907.10739 for a great introduction of 'decision understanding' vs. 'model understanding'
  + this tutorial focuses on explorative 'model understanding' via TX-Ray https://arxiv.org/abs/1912.00982

In [5]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from keras.datasets import imdb
import numpy as np
import warnings
from sklearn.metrics import accuracy_score
import sys
import random
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) # Keras IMDB data loading throws some warnings

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


# IMDB reviews as test bed
+ load Keras version of IMDB data
+ for some reason the word_id to word mapping is offset by -3. We define `turn_id_list_to_text_list_generator()` and `single_word_id_to_word()` to retrieve readable reviews and words from word-embedding id lists used for training 

In [6]:
def get_train_data_and_vocab_IMDB(padding_idx=0):
  """ Get the training set. Also get a map of embedding id to word indices for 
      later visualization 
      param: padding_idx = None, means do not pad. 0 or another number means use
      that embedding layer lookup id as the padding token - pytorch uses 0 by 
      default.
  """
  (X_train, y_train), (X_test, y_test) = imdb.load_data()
  emb_id_to_word_map = {emb_id:word for word, emb_id in imdb.get_word_index().items()}
  if padding_idx is not None:
    print('padding train inputs and vocab with word embedding at position', padding_idx)
    emb_id_to_word_map[0] = '<pad>'
    pad_sequences_inplace(X_train, padding_idx=padding_idx)
    pad_sequences_inplace(X_test, padding_idx=padding_idx)
  return X_train.tolist(), y_train.tolist(), X_test.tolist(), y_test.tolist(), emb_id_to_word_map

# The imdb dataset has an offseto of -3 for vocabulary and word_ids.
def single_word_id_to_word(emb_id_to_vocab_words, word_i, UNK_token='UNK'):
  return emb_id_to_vocab_words.get(word_id - 3, "<UNK>")

def turn_id_list_to_text_list_generator(emb_id_to_word_map, id_list, UNK_token='<UNK>'):
  print(type(id_list))
  if type(id_list[0]) is list: # list of list case - batch of text instances
    for row in id_list:
      yield [emb_id_to_vocab_words.get(word_id - 3, "<UNK>") for word_id in row]
    return # indicate iteration end, errors without
  # else case, single row/ text instance
  yield [emb_id_to_vocab_words.get(word_id - 3, "<UNK>") for word_id in id_list]

def pad_sequences_inplace(array, padding_idx=0):
      max_len = max([len(x) for x in array])
      print('max sequence len', max_len)
      for i in range(len(array)):
        if len(array[i]) < max_len:
          array[i] += [padding_idx]*(max_len - len(array[i]))
  # return array # actually an inplace action

def generate_batches(X, y, batchsize=None, max_num_batches=float('inf')):
  assert(len(X) == len(y))
  num_batches = 0
  for end_idx in range(batchsize, len(X)+batchsize, batchsize):
    num_batches += 1
    if num_batches <= max_num_batches:
      yield X[end_idx-batchsize:end_idx], y[end_idx-batchsize:end_idx]

X_train, y_train, X_test, y_test, emb_id_to_vocab_words = get_train_data_and_vocab_IMDB(padding_idx=0)

# test emb_id to text
text_rows_1and2 = [row for row in turn_id_list_to_text_list_generator(emb_id_to_vocab_words, X_train[:2])]
print("Example review. NOTE -3 in get() index\n", text_rows_1and2[0], '\n', text_rows_1and2[1] ) # the vocabcabulary index seems to be off set
# test batch generator batch sizes
print('Sizes of batches')
[(len(X_b), len(y_b)) for X_b, y_b in generate_batches(X_train[:10], y_train[:10], 4)]  # should be 4+4+2 for X and y

1646592/1641221 [==============================] - 0s 0us/step
padding train inputs and vocab with word embedding at position 0
max sequence len 2494
max sequence len 2315
<class 'list'>
Example review. NOTE -3 in get() index
 ['<UNK>', 'this', 'film', 'was', 'just', 'brilliant', 'casting', 'location', 'scenery', 'story', 'direction', "everyone's", 'really', 'suited', 'the', 'part', 'they', 'played', 'and', 'you', 'could', 'just', 'imagine', 'being', 'there', 'robert', "redford's", 'is', 'an', 'amazing', 'actor', 'and', 'now', 'the', 'same', 'being', 'director', "norman's", 'father', 'came', 'from', 'the', 'same', 'scottish', 'island', 'as', 'myself', 'so', 'i', 'loved', 'the', 'fact', 'there', 'was', 'a', 'real', 'connection', 'with', 'this', 'film', 'the', 'witty', 'remarks', 'throughout', 'the', 'film', 'were', 'great', 'it', 'was', 'just', 'brilliant', 'so', 'much', 'that', 'i', 'bought', 'the', 'film', 'as', 'soon', 'as', 'it', 'was', 'released', 'for', 'retail', 'and', 'would', '

[(4, 4), (4, 4), (2, 2)]

# Define swappable network components
+ each component can be called and swapped in a parameter conf dictionary below

### 1. define a simple classifier
+ this needs the input size from the convolution output - `sum(conf['filter_size_num'].values()) * conf['k_max']`
+ the output size is the number of classes - 1 for IMDB (binary prediction task)
+ optinally a two layer classfier can be used 

In [7]:
def classifier(conf):
  print('using 1 layer classifier')
  se = torch.nn.Sequential()
  if conf['clf_do_0'] is not None:
    se.add_module('clf_do_0', torch.nn.Dropout(p=conf['clf_do_0']))
  se.add_module('clf_0', nn.Linear(sum(conf['filter_size_num'].values()) * conf['k_max'], conf['clf_out_dim']))  # to 100
  out_dim = conf['clf_out_dim']
  return se, out_dim  # also return the dimension of the last layer output - not realy used in this tutorial

def two_layer_classifier(conf):
  print('using 1 layer classifier')
  se = torch.nn.Sequential()
  if conf['clf_do_0'] is not None:
    se.add_module('clf_do_0', torch.nn.Dropout(p=conf['clf_do_0']))
  se.add_module('clf_0', nn.Linear(sum(conf['filter_size_num'].values()) * conf['k_max'], 256))  # to 100
  if conf['clf_do_1'] is not None:
    se.add_module('clf_do_1', torch.nn.Dropout(p=conf['clf_do_1']))  
  se.add_module('clf_1', nn.Linear(256, conf['clf_out_dim']))  # to 100
  out_dim = conf['clf_out_dim']
  return se, out_dim  # also return the dimension of the last layer output - not realy used in this tutorial

2. we will use a config dictionary called `params` to define hyper parameters
  + this will make it easy to play with hyperparameters - i.e. less hard coding in many places/ cells

In [39]:
params = {'emb_dim': 100,
          'filter_size_num': {1: 10, 2: 7, 3: 3},  # e.g. {1: 5, 2: 3} conv window size 1: 5 filters, conv window size 2: 3 filters
          # 'filter_size_num': {1: 70, 2: 20, 3: 10},  # e.g. {1: 5, 2: 3} conv window size 1: 5 filters, conv window size 2: 3 filters
          'embs': 'random',  # optionally replace with pretrained embeddings. Embedding position must match word embedding id, e.g. word_id 10 -> [.1, .2, -.3,-.1]
          'loss_func': torch.nn.BCEWithLogitsLoss,
          'classifier_head': classifier, # or `two_layer_classifier`: is a funtion that returns a torch.nn.Sequential(). Dynamic classfier loading
          'clf_do_0': 0.2,  # helps stabilize training
          'clf_do_1': None, # for use with `two_layer_classifier`. Often only clf_do_0 is useful to avoid feature co-adaptations (results in instable training)
          'clf_out_dim': 1,  # the original TX-ray uses max pooling (k=1), but we may want to record the top-k activations
          'k_max': 1,  # to pool the top-k max values
          'vocab_size': len(emb_id_to_vocab_words),  # num of different words in the text, each word has one word embedding
          'pad_idx': 0,  # padding index is the indicates which embedding to use to indicate padding (no words). Typically word at embedding position 0 becomes a zero-vector. 
          'tune_embs': True,  # tune word embeddings or not. False lets the word embeddings stay random (Xavier initialized)
          'use_TXRAY_out': False,  # recode the network approach to collect TX-Ray input data
          'epochs': 20, # when to stop training
          'weight_decay': 2e-2,  # ADAM optimizer weight decay. default is 0 - see https://dejanbatanjac.github.io/2019/07/02/Impact-of-WD.html
          'lr': 0.0002,  # model learning rate
          'batch_size': 512,  # num samples in a micro batch
          'max_train_samples_to_use': None,  # limit number of train samples -- quick try outs. None means use all train samples
          'seed': 42,  # seed used for all random initializations
}
# hint
{1: 5, 1: 3, 3: 1} # NOTE: how using two convs with the same window size it not intended to be possible as it would break later activation extraction code

{1: 3, 3: 1}

In [20]:
class K_max_1D_Text_CNN(nn.Module):
  """ A text CNN with k-max pooling as in https://arxiv.org/pdf/1404.2188.pdf
      -- though here k-max is static.
  """
  
  def __init__(self, params, pretrained_embeddings=None): 
    # vocab_size, embs, emb_dim, num_filters, filter_sizes, classes, input_dropout, dropout):
    super().__init__()
    self.conf = params
    self.e = self.add_word_embeddding_layer(pretrained_embeddings)  # load either pretrained or random embeddings
    self.convs = self.conv1d_block(self.conf)
    self.loss = params['loss_func']
    self.classifier, self.out_dim = self.conf['classifier_head'](self.conf) # dynamically calls classfier construction according to param config. Expected to return  

  def add_word_embeddding_layer(self, pretrained_embeddings):
    embLUT = None
    # use random embeddings
    if self.conf['embs'] == 'random':  # pretrained_embeddings == None
      embLUT = nn.Embedding(num_embeddings=self.conf['vocab_size']+2,  # for some reason this vocab in IMDB has unmapped words, so reserve some extra
                            embedding_dim=self.conf['emb_dim'],
                            padding_idx=self.conf['pad_idx'])
    # load pretrained embeddings. Does not work well on IMDB dataset
    elif (type(pretrained_embeddings) == list) or (type(pretrained_embeddings) == np.ndarray):
      print("WARNING: on IMDB pretrained word embeddings are somewhat counterproductive")
      embLUT = nn.Embedding.from_pretrained(torch.FloatTensor(pretrained_embeddings),
                                            padding_idx=self.conf['pad_idx'],
                                            scale_grad_by_freq=self.conf.emb_grad_IDF_scaled)
    else:
        print(type(pretrained_embeddings))
        raise Exception('embeddings not in correct format')
    if self.conf['tune_embs'] is False:
        # https://discuss.pytorch.org/t/update-only-a-middle-layer-of-a-neural-network/35302
        self.freeze_parameters_and_track(embLUT)  
    return embLUT

  def freeze_parameters_and_track(self, layer):
    # https://discuss.pytorch.org/t/update-only-a-middle-layer-of-a-neural-network/35302
    # NOTE: no_grad() potentially turns off all gradients for layers before it is called,
    # so we use requires_grad=false
    for p in layer.parameters():
        p.requires_grad = False  # turn of tuning the net parameter

  def conv1d_block(self, conf):
    # constructs a 1D conf block, Each filter size can have a custom ammount of filters
    # e.g. conf = {'filter_size_num': {1:15, 2:10, 3:5}, 'emb_dim': 5, 'k_maxes': 2}
    in_channels = 1  # inputs is one channel 1D convs (for now)
    convs = nn.ModuleList()
    for win_size, out_channels in conf['filter_size_num'].items():
        kernel_size = (win_size, conf['emb_dim'])
        convs.append(nn.Conv2d(in_channels, out_channels, kernel_size))  # B, C_in = 1, F = num_filters, (K_W, K_H) = window_size = (wind_width, embedding dim)
    return convs
  
  def convolve_and_pool_single_view(self, em, k_max, record_activations=False, layer='cv_0'):
    """
    em: embeddings
    k_max: max k-pooling
    record_activations: for TX-Ray max activation visualization data recording
    layer: so we know which layer we are on
    """
    k_pooleded = []
    act_records = {layer: {}} if record_activations else None 
    for c in self.convs:
        # c() := B, F = num_filters, C_w = S - K_W + 1 = #times filter activates (max_pool_here) over seq dim, C_h = 1 = # time filter activates over word_emb dim (once = 1)
        # squeeze := B, F, C_W ; loose the folded to C_h = 1 emb_dim
        # kpool := B, F, K (reduces C_w to k max activations)
        cv = c(em).squeeze(3)
        assert cv.size()[-1] >= k_max, str(cv.size()[-1]) + " conv activations are too few for max-" + str(k_max) + ' pooling'  # ur input sequence is too short after n-gram convs
        topk, idx = cv.topk(k=k_max, dim=2)  # NOTE: K-MAX POOLING, where last dim is the conv outputs of each filter in cv
        if record_activations:
            conv_identifier = 'w=' + str(c.kernel_size[0])  # which conv bunch with a set window size
            act_records[layer][conv_identifier] = {'k_max_vals': topk, 'k_max_ids': idx}
        pool_acts = torch.relu(topk)  # B, F = num_filters of this conv c, K = max_k 
        flat_for_cat = torch.flatten(pool_acts, 1)  # B, F_i*K
        pooled_out = k_pooleded.append(flat_for_cat)
    extracted_features = torch.cat(k_pooleded, 1)  # B, F_i*K (variable) # concatenate over variable dim --> B, sum(F_i, K)(i=1 .. c)
    return extracted_features, act_records  # act_records is last is None if record_activations == False

  def forward(self, batch_data):
    """  `batch_data={'x_emb_ids':[[0, 1]], 'y_label_emb_ids':[[[3,4]]], 'y_yesNo_indicatior':[[1]]}`
    """
    # get sequence embeddings
    seq_em = self.e(batch_data)  # B, S, D = embedding_dim
    seq_em = seq_em.unsqueeze(1)  # B, C_in = 1 = #in channels, S, D
    pooled_features, _ = self.convolve_and_pool_single_view(em=seq_em,
                                                                  k_max=self.conf['k_max'],
                                                                  record_activations=self.conf['use_TXRAY_out'])
    pred_batch_logits = self.classifier(pooled_features)
    pred_batch_logits = pred_batch_logits.squeeze(-1)  # remove extra dim
    return pred_batch_logits  # indices tell us which convs were active

# Train the text CNN
+ stop on validation performance - manual, to try settings with
+ then run test set

In [40]:
def set_random_seed(seed):
  # Sets seed manually for both CPU and CUDA
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  # For atomic operations there is currently
  # no simple way to enforce determinism, as
  # the order of parallel operations is not known.
  # CUDNN
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False
  # System based
  random.seed(seed)
  np.random.seed(seed)

def make_input_tensor(input, device, Ttype=torch.float):
  # inputs are generally requires_grad False
  return torch.tensor(input, dtype=Ttype, requires_grad=False, device=device)

def save_model(model, params_dict, path='model.pt'):
  state = {
      'state_dict': model.state_dict(),
      'hyper_params': params_dict,
  }
  torch.save(state, path)
  print('model saved to:', path)

def load_model(model_class, path, device):
  state = torch.load(path)
  hyper_params = state['hyper_params']
  model = model_class(hyper_params)  # K_max_1D_Text_CNN(params)
  model.load_state_dict(state['state_dict'])
  model.to(device)
  return model, hyper_params

def eval(model, x, y, eval_vs_first_n_samples=5000):
  saved_mode = model.training
  # print('\nmodel was in train mode:', saved_mode)
  model.eval()
  x_eval = make_input_tensor(X_test[:eval_vs_first_n_samples], device, torch.long)
  y_eval = y_test[:eval_vs_first_n_samples]
  pseudo_probas = torch.sigmoid(model(x_eval)).detach().cpu().numpy()
  val_acc = accuracy_score(y_eval, pseudo_probas>0.5)
  print(', val acc:', val_acc)
  model.training = saved_mode # reset old mode
  return val_acc

def train_model(model, optim, params, X_train, y_train, model_save_path):
  # train loop
  model.train()  # enable gradient updates
  loss_fn = params['loss_func']()
  for epoch in range(params['epochs']):
    print('epoch', epoch)
    optim.zero_grad()  # reset gradients from prev epoch
    epoch_losses = []
    for X_b, y_b in generate_batches(X_train[:params['max_train_samples_to_use']], y_train[:params['max_train_samples_to_use']], params['batch_size']):
      x_b_t = make_input_tensor(X_b, device, torch.long)
      y_b_t = make_input_tensor(y_b, device, torch.float)
      logits = model(x_b_t)
      loss = loss_fn(logits, y_b_t) # if not BCEwithLogits expects (non-sigmoid) logits, vs (0,1) target labels
      loss.backward()  # compute gradients
      lozz = loss.item()  # .detach() loss from the computation graph and return loss value
      epoch_losses.append(lozz)
      optim.step()  # gradient update
      # break
      sys.stdout.write("\r" + str(lozz)) # replace line every time
      sys.stdout.flush()
    val_acc = eval(model, X_test, y_test, eval_vs_first_n_samples=5000)
    if val_acc > 0.85:
      break  # break when accuracy is 0.8 to not waste too much time on tuning
    sys.stdout.write("\repoch avg loss " + str(np.mean(epoch_losses)) + '\n') # replace line every time
    sys.stdout.flush()
  optim.zero_grad()  # reset gradients from prev epoch
  num_filterz = sum(params['filter_size_num'].values())
  model_save_path = model_save_path + str(num_filterz) + '_filters_' + str(val_acc)[:4] + '_val_acc.pt'
  save_model(model, params, path=model_save_path)

#@markdown train a model here, or load a trained one later. Default is `False`, to just work with already trained models, but you may want to train your own later on
train_new_model = False #@param ["True", "False"] {type:'raw'}
if train_new_model:
  set_random_seed(params['seed'])
  print("train model with params", params)
  text_CNN = K_max_1D_Text_CNN(params).to(device)  # to CPU or GPU memory
  optim = torch.optim.Adam(filter(lambda p: p.requires_grad, text_CNN.parameters()), lr=params['lr'], weight_decay=params['weight_decay'])
  print('using device', device, 'i.e. all model params on GPU', all([p.is_cuda for p in text_CNN.parameters()]))
  model_save_path = 'CNN_' # this is only the root file name. It will be extended
  train_model(text_CNN, optim, params, X_train, y_train, model_save_path)

train model with params {'emb_dim': 100, 'filter_size_num': {1: 10, 2: 7, 3: 3}, 'embs': 'random', 'loss_func': <class 'torch.nn.modules.loss.BCEWithLogitsLoss'>, 'classifier_head': <function classifier at 0x7f5730a53ea0>, 'clf_do_0': 0.2, 'clf_do_1': None, 'clf_out_dim': 1, 'k_max': 1, 'vocab_size': 88585, 'pad_idx': 0, 'tune_embs': True, 'use_TXRAY_out': False, 'epochs': 20, 'weight_decay': 0.02, 'lr': 0.0002, 'batch_size': 512, 'max_train_samples_to_use': None, 'seed': 42}
using 1 layer classifier
using device cuda i.e. all model params on GPU True
epoch 0
0.7498587965965271, val acc: 0.4856
epoch avg loss 0.734620561405104
epoch 1
0.6820360422134399, val acc: 0.5362
epoch avg loss 0.689798502289519
epoch 2
0.6693676114082336, val acc: 0.5942
epoch avg loss 0.6745076629580283
epoch 3
0.6522614359855652, val acc: 0.639
epoch avg loss 0.6545037748862286
epoch 4
0.6404529809951782, val acc: 0.6734
epoch avg loss 0.6376758801693819
epoch 5
0.6257920265197754, val acc: 0.684
epoch avg lo

# TX-Ray, aka activation maximization in a text CNN
+ basic steps below

### Goal
1. track words (n-grams) through the embedding and then convolutional layers into the pooling layers (see video below ↓)
  + https://drive.google.com/file/d/1aP0gV0Svv_BSfhR-RvyLuahxHTvAiWii/view?usp=sharing
2. assign the maximal activations to concrete words
3. collect the maximally activated words <u>per convolution filter</u> - 10 if a conv_layer has 10 filters
  + just use a `max_acts = {'conv_0':{'like':list_of_max_activations_over_a_batch_or_dataset}}`, where `list_of_max_activations_over_a_batch_or_dataset = [.1, .5 ..., .9]`
4. visualize any convolution filter via plotly (neuron feature preference view)
  + compute average activation per feature in a filter, e.g. for the feature `like` in `conv_0` `max_acts['conv_0']['like'] = np.mean(max_acts['conv_0']['like'])` # done inplace to save Colab RAM

In [2]:
#@markdown Collecting activations using forward hooks concept 
from IPython.display import HTML
HTML("""
<video width=800 controls>
      <source src="https://drive.google.com/uc?id=1aP0gV0Svv_BSfhR-RvyLuahxHTvAiWii" type="video/mp4">
</video>
""") 

## Use pytorch forward hooks to get intermediate activations
+ [Concept used in TX-Ray: forward hooks to get activations](http://web.stanford.edu/~nanbhas/blog/forward-hook.html) - by Nandita Bhaskhar
+ [Advanced examples: foward and backward hooks + use cases examples](https://blog.paperspace.com/pytorch-hooks-gradient-clipping-debugging/) - by AYOOSH KATHURIA

In [43]:
# TXacts = get_activations_via_forward_hook(text_CNN, xl)
def forward_hook_with_named_activation(act_name, activations):
  """ use a closure (function in function), to define the hook, and name the stored result)
      Fill activations dict in place using act_name as a key to distinguish layers - name act_name as you like.
  """
  # input is a tuple of packed inputs
  # output is a Tensor. output.data is the Tensor we are interested
  def hook_fn(component, input, output):  
    activations[act_name] = output.detach() # detach, to get the data
  return hook_fn  # this function 'pointer' is what .register_forward_hook(hook) requires as its parameter hook_fn()

def get_activations_via_forward_hook(model, input_data):
  activations = {}
  hook_fns = [] 
  for c in range(len(text_CNN.convs)):
    hook_fns.append(model.convs[c].register_forward_hook(forward_hook_with_named_activation('conv_' + str(c + 1), # filter names are +1
                                                                                               activations)))
  _ = model(input_data)  # This model.forward(input_data) calls the hooks
  for h in hook_fns:
    h.remove()  # release hook functions, otherwise we get buggy behaviour -- e.g. older hooks may remain in future call
  return activations

### Example to understand activation collection

In [44]:
def example_top_k_index_and_gather():
  num_words_in_sequence = 4
  a = torch.randn(2, 3, num_words_in_sequence)
  print(a.size())
  dim_to_max_pool_over = 2  # pooling dimension, gets folded/ collapsed to 1
  max_2_pool = 2
  mx, mi = torch.topk(a, k=max_2_pool, dim=dim_to_max_pool_over)  # get max values and max indices (positions)
  print("max value size", mx.size(), 'max value index size (batch_size, ... , k from max-k-pool at the dim specified by dim_to_max_pool_over))', mi.size())
  print('max ids', mi)
  max_vals_via_gather_index_syntax = a.gather(dim_to_max_pool_over, mi)
  print(mx == max_vals_via_gather_index_syntax) # should be all True, to show that topk-maxes and gater(topk-indices) maxes are the same
example_top_k_index_and_gather()


torch.Size([2, 3, 4])
max value size torch.Size([2, 3, 2]) max value index size (batch_size, ... , k from max-k-pool at the dim specified by dim_to_max_pool_over)) torch.Size([2, 3, 2])
max ids tensor([[[2, 0],
         [1, 0],
         [3, 1]],

        [[0, 1],
         [3, 2],
         [0, 1]]])
tensor([[[True, True],
         [True, True],
         [True, True]],

        [[True, True],
         [True, True],
         [True, True]]])


In [90]:
#@title (A) use the model to collect its activations. Train one above, or load a previously trained one -- see the data folder.
#@markdown NOTE: when load a previous model, this also overloads the `params` dict to avoid invalid states
load_previously_trained_model = True #@param ["False", "True"] {type:"raw"}
model_path_to_load = "/content/model_10_filters_83pc_val_acc.pt" #@param ["model.pt", "/content/model_80pc_val_acc.pt", "/content/model_10_filters_83pc_val_acc.pt", "/content/model_100_filters_82pc_val_acc.pt", "/content/you_model."]
if load_previously_trained_model:
  print('loading model to', device)
  text_CNN, params = load_model(model_class=K_max_1D_Text_CNN, path=model_path_to_load, device=device)
  print('loaded params\n:', params)

#@title list available module names within the model
for name, module in text_CNN.named_modules():
  print(name) # layers in ModuleLists are addressed via [i], e.g. text_CNN.convs[0]
for c in range(len(text_CNN.convs)):
  print(text_CNN.convs[c])

# collect activations for TX-Ray -- maximal activations
#@markdown you could also collect activations on the test set X_test, y_test
def collect_activations(X, y, model, params, device, max_num_batches, k_from_max_k_pooling):
  max_activations_dict = {'conv_' + str(conv_i):{'filter_size':conv_i, 'filters':{f:dict() for f in range(num_filters)}} for conv_i, num_filters in params['filter_size_num'].items()}
  print("activation map structure\n", max_activations_dict)  # the empty activations dictionary
  model_state = model.training
  model.eval()
  for X_b, _ in generate_batches(X[:params['max_train_samples_to_use']], y[:params['max_train_samples_to_use']],
                                 params['batch_size'], max_num_batches=max_num_batches):
    input_batch = make_input_tensor(X_b, device, torch.long)
    try:
      named_batch_conf_activations = get_activations_via_forward_hook(model, input_batch)
    except:
      input_batch
    sequence_activation_dim = 2  # conv activations are of size (batch_size, cnn_filters_num, sequence_length,)
    for conv_block in max_activations_dict.keys():
      max_values, max_ids = torch.topk(named_batch_conf_activations[conv_block], k=k_from_max_k_pooling, dim=sequence_activation_dim)
      # max_ids are of size (batch_size, cnn_filters_num, k_from_max_k_pooling, 1)
      # print(max_ids.size())
      for row in range(input_batch.size(0)): # batch size may not be the same for the last batch
        # print(input_batch[row])
        # print(max_values[row].squeeze(-1).tolist())
        # print(max_ids[row].squeeze(-1).tolist())
        for cnn_filter in range(len(max_ids[row])):
          # print('\nFilter', cnn_filter)
          max_activation_ids = input_batch[row].gather(0, max_ids[row][cnn_filter].squeeze(-1)).tolist() 
          max_activation_vals = max_values[row][cnn_filter].squeeze(-1).tolist()
          assert(len(max_activation_vals) == len(max_activation_ids))
          # print('argmax || indexed max_word_ids\n', max_ids[row][cnn_filter].squeeze(-1), '||', max_activation_ids)
          # print('k max filter-on-word activations', max_activation_vals)
          for i in range(len(max_activation_ids)):
            if max_activation_ids[i] in max_activations_dict[conv_block]['filters'][cnn_filter]: # feature occured before in this filter
              max_activations_dict[conv_block]['filters'][cnn_filter][max_activation_ids[i]].append(max_activation_vals[i])  # add to activation list
            else:
              max_activations_dict[conv_block]['filters'][cnn_filter][max_activation_ids[i]] = [max_activation_vals[i]] # add new activation list
          # break
        # break
      # break    # uncomment print and 3 breaks above to see how this collection of filter activations works
    # break
  model.training = model_state # return model state
  return max_activations_dict


#@markdown + change the amount of data to use for collecting max-activations. To use all training data set `False`
batch_size = 128  #@param {type: "number"}
params['batch_size'] = batch_size  # lower for debugging
max_num_batches =    200#@param {type: "number"}  float('inf') # inf meaning as many a are in the dataset
k_from_max_k_pooling = 1 #@param {type: "number"}
                         # how many max activations to collect this can be different from params['k-max']
max_activations_dict1 = collect_activations(X_train, # can be swithed for test data 
                                            y_train,
                                            model=text_CNN,
                                            params=params,
                                            device=device,
                                            max_num_batches=max_num_batches,
                                            k_from_max_k_pooling=k_from_max_k_pooling)

loading model to cuda
using 1 layer classifier
loaded params
: {'emb_dim': 100, 'filter_size_num': {1: 10, 2: 7, 3: 3}, 'embs': 'random', 'loss_func': <class 'torch.nn.modules.loss.BCEWithLogitsLoss'>, 'classifier_head': <function classifier at 0x7f5730a53ea0>, 'clf_do_0': 0.2, 'clf_do_1': None, 'clf_out_dim': 1, 'k_max': 1, 'vocab_size': 88585, 'pad_idx': 0, 'tune_embs': True, 'use_TXRAY_out': False, 'epochs': 20, 'weight_decay': 0, 'lr': 0.0002, 'batch_size': 128, 'max_train_samples_to_use': None, 'seed': 42}

e
convs
convs.0
convs.1
convs.2
classifier
classifier.clf_do_0
classifier.clf_0
Conv2d(1, 10, kernel_size=(1, 100), stride=(1, 1))
Conv2d(1, 7, kernel_size=(2, 100), stride=(1, 1))
Conv2d(1, 3, kernel_size=(3, 100), stride=(1, 1))
activation map structure
 {'conv_1': {'filter_size': 1, 'filters': {0: {}, 1: {}, 2: {}, 3: {}, 4: {}, 5: {}, 6: {}, 7: {}, 8: {}, 9: {}}}, 'conv_2': {'filter_size': 2, 'filters': {0: {}, 1: {}, 2: {}, 3: {}, 4: {}, 5: {}, 6: {}}}, 'conv_3': {'filter_

# Activation visualization

In [51]:
from IPython.display import HTML
import plotly.graph_objects as go

## ANALYSIS 1: Plot number of unique features per cnn filter

In [91]:
#@title Check the structure of activation recodings for (TX-ray like) analysis
print(max_activations_dict1.keys()), print(max_activations_dict1['conv_1'].keys()), print(max_activations_dict1['conv_1']['filters'].keys())

def get_number_of_unique_features_in_a_filter(max_act_dict, layer_name='conv_1', filter_num=0, verbose=False):
  if verbose:
    print('availabel_layers', max_act_dict.keys())  # print what layers (filters/params/neurons) in them can visualized
  return len(max_act_dict[layer_name]['filters'][filter_num].keys())

plot_color = 'lightseagreen' #@param ['lightblue', 'lightcoral', 'lightcyan', 'lightgoldenrodyellow', 'lightgray', 'lightgrey', 'lightgreen', 'lightpink', 'lightsalmon', 'lightseagreen', ''lightskyblue', 'lightslategray', 'lightslategrey', 'lightsteelblue', 'lightyellow', 'lime', 'limegreen']
def plot_bar(x, y, title, x_title, y_title, width=600, height=300, color='lightblue'):
  fig = go.Figure(data=go.Bar(x=x, y=y, marker_color=color))
  fig.update_layout(title_text=title,
                    yaxis=dict(title=y_title),
                    xaxis=dict(title=x_title),
                    width=width,
                    height=height)
  fig.show()

#@markdown + Amount of unique features per filter -- not all filters activate on the same amount of features
def visualize_unique_features_in_filters(max_activations_dict_i, color):
  for conv in max_activations_dict_i.keys():
    x_filter_num = []
    y_unique_features_in_filter = []
    for filter_n in max_activations_dict_i[conv]['filters'].keys():
      unique_features_in_filter = get_number_of_unique_features_in_a_filter(max_activations_dict_i, layer_name=conv, filter_num=filter_n)
      x_filter_num.append(filter_n)
      y_unique_features_in_filter.append(unique_features_in_filter)
    print(x_filter_num, y_unique_features_in_filter)
    plot_bar(x=x_filter_num, y=y_unique_features_in_filter, x_title='filter', y_title='#num unique features',
            title=conv + " unique features per filter", color=color)
    
visualize_unique_features_in_filters(max_activations_dict1, color=plot_color)

dict_keys(['conv_1', 'conv_2', 'conv_3'])
dict_keys(['filter_size', 'filters'])
dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9] [4091, 3475, 2962, 2573, 3276, 3896, 3373, 3502, 3851, 2946]


[0, 1, 2, 3, 4, 5, 6] [4713, 3554, 4079, 1704, 3903, 4726, 3442]


[0, 1, 2] [6307, 4591, 4249]


### <font color='Gold'>Familiarization exercise 1: rerun activation collection for other models </font>
1. chose parameters on the right and rerun `collect_activations()` 
2. also rerun activation collection `visualize_unique_features_in_filters()`
+ what changes?
+ why does it change? 
+ does it matter:
  + over how much data we collect activations? explore `max_num_batches` setting
  + how many max_activations we collect? `k_from_max_k_pooling`
+ EXTRA: consider training your own model with your own `params` dict configurations, then load the new model path


In [92]:
#@title explore parameters influence on uniqueness of filters
load_previously_trained_model = True #@param ["False", "True"] {type:"raw"}
model_path_to_load = "/content/model_100_filters_82pc_val_acc.pt" #@param ["/content/your_model.pt", "/content/model_10_filters_83pc_val_acc.pt", "/content/model_100_filters_82pc_val_acc.pt"]
if load_previously_trained_model:
  print('loading model to', device)
  text_CNN, params = load_model(model_class=K_max_1D_Text_CNN, path=model_path_to_load, device=device)
  print('loaded params\n:', params)

#@markdown + change the amount of data to use for collecting max-activations.
batch_size =   16#@param {type: "number"}
params['batch_size'] = batch_size  # lower for debugging
max_num_batches =    5#@param {type: "number"}  float('inf') # inf meaning as many a are in the dataset
k_from_max_k_pooling =  1#@param {type: "number"}
                         # how many max activations to collect this can be different from params['k-max']
#@markdown 1. collect activations
max_activations_dict2 = collect_activations(X_train, # can be swithed for test data 
                                            y_train,
                                            model=text_CNN,
                                            params=params,
                                            device=device,
                                            max_num_batches=max_num_batches,
                                            k_from_max_k_pooling=k_from_max_k_pooling)

color = 'lightcoral' #@param ['lightblue', 'lightcoral', 'lightcyan', 'lightgoldenrodyellow', 'lightgray', 'lightgrey', 'lightgreen', 'lightpink', 'lightsalmon', 'lightseagreen', ''lightskyblue', 'lightslategray', 'lightslategrey', 'lightsteelblue', 'lightyellow', 'lime', 'limegreen']
#@markdown 1. visualize number of unique features
visualize_unique_features_in_filters(max_activations_dict2, color=color)

#@markdown **EXTRA** train or load your own cnn
# params['filter_size_num'] = {1:300, 5:10}
# model_save_path = 'my_model.pt'
# # train_model(text_CNN, optim, params, X_train, y_train, model_save_path)
# max_activations_dict1 ...
# visualize_unique_features_in_filters ...


loading model to cuda
using 1 layer classifier
loaded params
: {'emb_dim': 100, 'filter_size_num': {1: 70, 2: 20, 3: 10}, 'embs': 'random', 'loss_func': <class 'torch.nn.modules.loss.BCEWithLogitsLoss'>, 'classifier_head': <function classifier at 0x7f5730a53ea0>, 'clf_do_0': 0.2, 'clf_do_1': None, 'clf_out_dim': 1, 'k_max': 1, 'vocab_size': 88585, 'pad_idx': 0, 'tune_embs': True, 'use_TXRAY_out': False, 'epochs': 20, 'weight_decay': 0, 'lr': 0.0002, 'batch_size': 512, 'max_train_samples_to_use': None, 'seed': 42}
activation map structure
 {'conv_1': {'filter_size': 1, 'filters': {0: {}, 1: {}, 2: {}, 3: {}, 4: {}, 5: {}, 6: {}, 7: {}, 8: {}, 9: {}, 10: {}, 11: {}, 12: {}, 13: {}, 14: {}, 15: {}, 16: {}, 17: {}, 18: {}, 19: {}, 20: {}, 21: {}, 22: {}, 23: {}, 24: {}, 25: {}, 26: {}, 27: {}, 28: {}, 29: {}, 30: {}, 31: {}, 32: {}, 33: {}, 34: {}, 35: {}, 36: {}, 37: {}, 38: {}, 39: {}, 40: {}, 41: {}, 42: {}, 43: {}, 44: {}, 45: {}, 46: {}, 47: {}, 48: {}, 49: {}, 50: {}, 51: {}, 52: {},

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19] [65, 51, 58, 64, 34, 58, 69, 61, 49, 64, 63, 50, 57, 73, 61, 63, 50, 62, 71, 47]


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9] [67, 71, 70, 61, 73, 57, 76, 67, 69, 59]


## Visualize maximal activation behaviour per filter 
Explore activation behaviour to find starting points for deeper analysis towards better 'model understanding'.
### <font color='Gold'>Familiarization exercise 2: explore max activation statistics </font>
+ try out different settings for `filter_activation_statistic_to_plot` and rerun `visualize_filter_activation_statistics` multiple times (with differen colors, models, data amounts etc.)
+ **TIP:** add your personal answers here or make new cells to record insights
  + what do the results mean -- what questions can we discover and explore?
  + what filter may be more useful vs. noisy?
  + are all filters actually used -- are some over/ under used?


In [85]:
#@title Visualize filter k-max activation statistics
def get_activation_mass_per_filter(max_act_dict, layer_name='conv_1', filter_num=0, verbose=False):
  activations_distribution = [item for sublist in max_act_dict[layer_name]['filters'][filter_num].values() for item in sublist] # flatten list[lists] to list
  return {'sum': sum(activations_distribution),
          'min': min(activations_distribution),
          'max': max(activations_distribution),
          'mean': np.mean(np.array(activations_distribution)),
          'std': np.std(np.array(activations_distribution)),
          'act_dist': activations_distribution} # activations wont work with the bar plot.
          #@markdown (optional) #TODO rewrite code to plot `activation_distribution` per filter

def visualize_filter_activation_statistics(max_activations_dict, filter_activation_stats_to_plot, color):
  for conv in max_activations_dict.keys():
    x_filter_num = []
    stat_per_filter = {'sum':[], 'min':[], 'max':[], 'mean':[], 'std':[]}
    for filter_n in max_activations_dict[conv]['filters'].keys():
      actication_stats = get_activation_mass_per_filter(max_activations_dict, layer_name=conv, filter_num=filter_n)
      stat_per_filter[filter_activation_stats_to_plot].append(actication_stats[filter_activation_stats_to_plot])
      x_filter_num.append(filter_n)
    # print(x_filter_num, stat_per_filter)
    plot_bar(x=x_filter_num, y=stat_per_filter[filter_activation_stats_to_plot], 
            x_title='filter',
            y_title=filter_activation_stats_to_plot + ' activation',
            title=conv + " " + filter_activation_stats_to_plot + " activations per filter",
            color=color)

#@title explore parameters influence on uniqueness of filters
load_previously_trained_model = True #@param ["False", "True"] {type:"raw"}
model_path_to_load = "/content/model_100_filters_82pc_val_acc.pt" #@param ["/content/model_10_filters_83pc_val_acc.pt", "/content/model_100_filters_82pc_val_acc.pt", "/content/you_model."]
if load_previously_trained_model:
  print('loading model to', device)
  text_CNN, params = load_model(model_class=K_max_1D_Text_CNN, path=model_path_to_load, device=device)
  print('loaded params\n:', params)

#@markdown + change the amount of data to use for collecting max-activations.
batch_size =   256#@param {type: "number"}
params['batch_size'] = batch_size  # lower for debugging
max_num_batches =    200#@param {type: "number"}  float('inf') # inf meaning as many a are in the dataset
k_from_max_k_pooling = 1 #@param {type: "number"}
                         # how many max activations to collect this can be different from params['k-max']

#@markdown </br></br></br></br></br></br></br></br></br></br></br></br></br></br></br></br></br>
#@markdown <font color='lawngreen'>HINT</font>:</p>
#@markdown + re-collect activations: takes a moment. Set to `False` if you just want to revisualize via `filter_activation_statistic_to_plot` modes below
#@markdown + the more batches of activation you collect, the more meaningful the plots (but this becomes slow - so use `False` after collecting once)
recollect_activations = False #@param ['True', 'False'] {type:'raw'}
if recollect_activations:
  print('collecting activations')
  max_activations_dict2 = collect_activations(X_train, # can be swithed for test data 
                                              y_train,
                                              model=text_CNN,
                                              params=params,
                                              device=device,
                                              max_num_batches=max_num_batches,
                                              k_from_max_k_pooling=k_from_max_k_pooling)

#@markdown <font color='Gold'> Exercise: analyse activation statisitic of each filter </font>
color = 'lightslategray' #@param ['lightblue', 'lightcoral', 'lightcyan', 'lightgoldenrodyellow', 'lightgray', 'lightgrey', 'lightgreen', 'lightpink', 'lightsalmon', 'lightseagreen', ''lightskyblue', 'lightslategray', 'lightslategrey', 'lightsteelblue', 'lightyellow', 'lime', 'limegreen']
#@markdown <font color='Gold'>What do the different modes tell us?</font> Does this depend the amount of collected activations?
filter_activation_statistic_to_plot = 'sum' #@param ['sum','min', 'max', 'mean', 'std']

# rerun this multiple times for sum, min, models etc
visualize_filter_activation_statistics(max_activations_dict2, filter_activation_statistic_to_plot, color)

# copy paste some code to plot multiple settings for comparison
# visualize_filter_activation_statistics(max_activations_dict2, 'std', 'lightskyblue')
# visualize_filter_activation_statistics(max_activations_dict_more_k_max, 'std', 'lightblue') # TODO
# visualize_filter_activation_statistics(max_activations_dict_smaller_model, 'max', 'lightgray') # TODO

loading model to cuda
using 1 layer classifier
loaded params
: {'emb_dim': 100, 'filter_size_num': {1: 70, 2: 20, 3: 10}, 'embs': 'random', 'loss_func': <class 'torch.nn.modules.loss.BCEWithLogitsLoss'>, 'classifier_head': <function classifier at 0x7f5730a53ea0>, 'clf_do_0': 0.2, 'clf_do_1': None, 'clf_out_dim': 1, 'k_max': 1, 'vocab_size': 88585, 'pad_idx': 0, 'tune_embs': True, 'use_TXRAY_out': False, 'epochs': 20, 'weight_decay': 0, 'lr': 0.0002, 'batch_size': 512, 'max_train_samples_to_use': None, 'seed': 42}


#<font color='orange'>Advanced optional self-exercises below</font>
+ What does it mean, when some filters do not activate or activate a lot?
  + 🤔 do we need them or should we prune some filters them? - <font color='orange'>(Opt-Ex1: inline pruning)</font>
  + 🤔 are we overparameterized?
    + with more/ less CNN filters, how do distribitions look after training? - <font color='orange'>(Opt-Ex2: influence of filter amount)</font>
      + retrain net, with more or less filter - TIP: may need more/ less learning rate and dropout
+ Are some filters redundant:
  + how unique is each filter in terms of its maximally activated features - <font color='orange'>(Opt-Ex3: Filter redundancy)</font>

## <font color='orange'>Opt-EX1:</font> advanced, open-ended tasks: <font color='orange'>inline pruning</font>
1. think about how you could identify unneeded convolution filters and play with filters that one could reason/ experiment with as:
  + noisy filters
  + overspecialized filters (e.g. seldomly used filters)
  + overly general filters (e.g overused filters)
  + any criterion that came out of your previous analysis

## Approach outline
+ use forward hooks to zero-out convolution filters
  + see `forward_hook_with_named_activation()` on how to extract activations. Now you want to manipulate the output
  + https://discuss.pytorch.org/t/modify-the-feature-activation-maps/56276, in old pytorch versions <1.2, on could use pre-hooks. Which is cumbersome. In >=Pytroch 1.2 forward hooks can manipulate the output they are getting 
  + see 
+ <font color='lime'>HINT:</font> read the paper https://arxiv.org/pdf/1912.00982.pdf Table 2 if you need inspiration for XAI based pruning.

In [86]:
#@title <font color='turquoise'>Opt-EX1 solution hints:</font> A minimum example of how to inline prune with forward hooks in pytorch>=1.2
#@markdown Approach: code has to be heavily adapted for the CNN - see activation extraction code in `collect_activations()` and use hooks to prune inline instead of collecting activations. Code-hints for simple 2 layer net below. 

#@markdown 1. make a simple 2 layer net
net = nn.Sequential(nn.Linear(7,4), nn.Linear(4, 7))
print(net)
x = torch.randn(7)

#@markdown 2. **prune inline:** define a masking hook 'closure' `mask_activations_to_zero()`, to pass a mask to your `hook_fn` (optinally track activations)
def mask_activations_to_zero(mask, mask_val, activations=None):
  """ To record 'activations', pass in a empty dict """
  # in you implementation, you can use activations to check for bugs while coding, but do not need them to predict
  def hook_fn(module, inputs, outputs):
    if not module.training:  # make sure this is only usable in .train() mode
      activations['in'] = inputs[0].detach().tolist() # [0] is the input vals in a tuple
      activations['out'] = outputs.detach().tolist()  # to list, to avoid side effects from masking, i.e. otherwise 'out' and 'pruned' point to the same tensor
      #@markdown 3. prune inline `outputs[mask] = mask_val`, activation recording is not needed in your code
      outputs[mask] = mask_val # inline pruning
      activations['pruned'] = outputs.detach().tolist()
    else:
      print('modle not in model.eval() mode. So, not using hook')
  return hook_fn  #@markdown 4. return the `hook_fn` object to provide .register_forward_hook(hook_fn) the hook_fn it expects

prune_mask_val = 0 #@param [0, -1] {type:"raw"}
prune_mask = [5,6] #@markdown 5. define an activation (outputs) mask, e.g. `prune_mask = [5,6]` to 'prune' the 6-7th activation
# or a negative value, e.g. (-1) before max-pooling, to ensure the activations are removed by max_pooling
record_of_IO_and_pruned_activations = dict()
# in size=(4), out size(7), prune the last two positions [5,6]
hook = net[1].register_forward_hook(mask_activations_to_zero(prune_mask, prune_mask_val, activations=record_of_IO_and_pruned_activations))
net.eval()
net(x)
#@markdown 6. see masking/ inline pruning in action
for item in record_of_IO_and_pruned_activations.items():
  print(item)


#@markdown 7. **TODO:** adjust code for your model (e.g. text_CNN). Also see activation extraction code in previous cells to get the CNN layer outputs
def eval_with_hooks(model, x, y, eval_vs_first_n_samples=5000):
  saved_mode = model.training
  model.eval()
  # normal validation/ test accuracy
  x_eval = make_input_tensor(X_test[:eval_vs_first_n_samples], device, torch.long)
  y_eval = y_test[:eval_vs_first_n_samples]
  pseudo_probas = torch.sigmoid(model(x_eval)).detach().cpu().numpy()
  val_acc = accuracy_score(y_eval, pseudo_probas>0.5)
  print(', val acc:', val_acc)
  
  #TO_IMPLEMENT: forward_hooks that 0 out or set activations to negative, so that max pooling ignores them 
  # 

  model.training = saved_mode # reset old mode
  return val_acc, pruned_val_acc
# eval_with_hooks(text_CNN ... )

Sequential(
  (0): Linear(in_features=7, out_features=4, bias=True)
  (1): Linear(in_features=4, out_features=7, bias=True)
)
('in', [-0.4724668264389038, 0.8964922428131104, -0.12034046649932861, 1.1145015954971313])
('out', [0.44613146781921387, -0.43000528216362, -1.0070600509643555, 0.2327728271484375, 0.8358603715896606, 0.5459489226341248, -0.6198179125785828])
('pruned', [0.44613146781921387, -0.43000528216362, -1.0070600509643555, 0.2327728271484375, 0.8358603715896606, 0.0, 0.0])



## <font color='orange'>Opt-EX2:</font> adv., optional task:  <font color='orange'>Influence of filter amount (overparameterization)</font>
+ how does CNN filter preference (max-activations) depend on the number of filters? Exploring this task will also be useful for Opt-Ex3 -- i.e. use a wide net = many CNN filters

## Approach outline
+ train a CNN with more or less filters, e.g. `params['filter_size_num']` = {1: 70, 2: 20, 3: 10} vs. {1: 5, 2: 3, 3: 2}
  + TIP: download `model_10_filters_83pc_val_acc.pt` and `model_10_filters_83pc_val_acc.pt` from https://github.com/copenlu/ALPS_2021 and upload them into the data folder (left pane, here) - then right-click->copy_path to get the path for `load_model()` 


In [93]:
# code here
#@markdown use the pretrained models from https://github.com/copenlu/ALPS_2021/trained_models, or train even larger ones using `train_models(params)`, where params is configured for more filters etc.

## <font color='orange'>Opt-Ex3: Filter redundancy</font>
+ are the max-activated features in each filter unique?
### Approach outline
  + modify `get_number_of_unique_features_in_a_filter` to return the set of feature indices (e.g. `get_unique_feature_set_of_a_filter`) for each filter in a convolution block - then compare the indices
  + the more filters a model has, the more chance for redundancy? -- again load other models by copying code from above cells

In [87]:
def get_unique_feature_set_of_a_filter(max_act_dict, layer_name='conv_1', filter_num=0):
  return set(max_activations_dict[layer_name]['filters'][filter_num].keys())

#@markdown + TODO improve method if need be
def get_unique_in_filters(max_activations_dict, color):
  for conv in max_activations_dict.keys():
    x_filter_num = []
    y_unique_features_in_filter = []
    for filter_n in max_activations_dict[conv]['filters'].keys():
      unique_features_in_filter = get_unique_feature_set_of_a_filter(max_activations_dict, layer_name=conv, filter_num=filter_n)
      x_filter_num.append(filter_n)
      y_unique_features_in_filter.append(unique_features_in_filter)
  return y_unique_features_in_filter, x_filter_num  # list of set(unique_features_in_a_filter)

#@markdown + code filter feature overlap (redundany) - e.g. jaccard similarity
#@markdown + you may also want to consider how strongly the filters activate (see code in <font color='Gold'>Familiarization exercise 2:</font>)